In [332]:
import numpy as np

In [333]:
import pandas as pd

In [334]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,f_regression
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression

In [335]:
df=pd.read_csv('CreditCard_DV.csv')

In [336]:
df

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Income_Category,Credit_Limit,Total_Revolving_Bal,Avg_Purchase,Total_Trans_Amt
0,768805383,Existing Customer,45,M,3,High School,$60K - $80K,12691.0,777,11914.0,1144
1,818770008,Existing Customer,49,F,5,Graduate,Less than $40K,8256.0,864,7392.0,1291
2,713982108,Existing Customer,51,M,3,Graduate,$80K - $120K,3418.0,0,3418.0,1887
3,769911858,Existing Customer,40,F,4,High School,Less than $40K,3313.0,2517,796.0,1171
4,709106358,Existing Customer,40,M,3,Uneducated,$60K - $80K,4716.0,0,4716.0,816
...,...,...,...,...,...,...,...,...,...,...,...
95,719712633,Existing Customer,64,M,1,Graduate,Less than $40K,1709.0,895,814.0,1673
96,772629333,Existing Customer,45,M,3,Graduate,$40K - $60K,3454.0,1200,2254.0,1313
97,720336708,Existing Customer,53,M,3,Doctorate,$40K - $60K,3789.0,1706,2083.0,1609
98,802013583,Existing Customer,56,M,3,College,$120K +,9689.0,2250,7439.0,1158


In [337]:
df.drop(['CLIENTNUM','Attrition_Flag','Income_Category'],axis=1,inplace=True)

In [338]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Customer_Age         100 non-null    int64  
 1   Gender               100 non-null    object 
 2   Dependent_count      100 non-null    int64  
 3   Education_Level      100 non-null    object 
 4   Credit_Limit         100 non-null    float64
 5   Total_Revolving_Bal  100 non-null    int64  
 6   Avg_Purchase         100 non-null    float64
 7   Total_Trans_Amt      100 non-null    int64  
dtypes: float64(2), int64(4), object(2)
memory usage: 6.4+ KB


In [339]:
df.head()

,Customer_Age,Gender,Dependent_count,Education_Level,Credit_Limit,Total_Revolving_Bal,Avg_Purchase,Total_Trans_Amt
0,45,M,3,High School,12691.0,777,11914.0,1144
1,49,F,5,Graduate,8256.0,864,7392.0,1291
2,51,M,3,Graduate,3418.0,0,3418.0,1887
3,40,F,4,High School,3313.0,2517,796.0,1171
4,40,M,3,Uneducated,4716.0,0,4716.0,816


In [340]:
x=df.drop(['Total_Trans_Amt'],axis=1)
x['Gender'].unique()

array(['M', 'F'], dtype=object)

In [367]:
y=df['Total_Trans_Amt']
y

0     1144
1     1291
2     1887
3     1171
4      816
      ... 
95    1673
96    1313
97    1609
98    1158
99     836
Name: Total_Trans_Amt, Length: 100, dtype: int64

In [368]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [369]:
x_train

,Customer_Age,Gender,Dependent_count,Education_Level,Credit_Limit,Total_Revolving_Bal,Avg_Purchase
55,49,F,4,Graduate,3298.0,2200,1098.0
88,44,M,3,High School,12756.0,837,11919.0
26,59,M,1,High School,14784.0,1374,13410.0
42,50,F,1,College,17304.0,2517,14787.0
69,50,M,2,Doctorate,25300.0,1330,23970.0
...,...,...,...,...,...,...,...
60,55,M,3,Unknown,3436.0,2016,1420.0
71,55,F,2,Graduate,2216.0,1034,1182.0
14,57,F,2,Graduate,2436.0,680,1756.0
92,45,M,1,Graduate,6576.0,0,6576.0


In [370]:
y_train

55    1052
88    1413
26    1197
42    1756
69     837
      ... 
60    1097
71    1540
14    1570
92    1465
51     705
Name: Total_Trans_Amt, Length: 80, dtype: int64

In [371]:
trf1=ColumnTransformer([
    ('tr1',OneHotEncoder(drop='first', sparse=False),[1])
                    ],remainder='passthrough')

In [372]:
trf2=ColumnTransformer([
    (('tr1'),LabelEncoder(),[3])
    ],remainder='passthrough')


In [373]:
trf3=ColumnTransformer([(('tr1'),MinMaxScaler(),[4,5,6])]
                      ,remainder='passthrough')

In [374]:
trf4 = SelectKBest(score_func=f_regression,k=5)

In [375]:
trf5= LinearRegression()

In [376]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)])

In [377]:
from sklearn import set_config
set_config(display='diagram')

In [378]:
pipe.fit(x_train,y_train)

TypeError: fit_transform() takes 2 positional arguments but 3 were given